In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source=dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date=dbutils.widgets.get("p_file_date")

#### Define Schema

In [0]:
from pyspark.sql.types import StructType,StringType,StructField,IntegerType,DateType
name_schema=StructType(fields=[StructField("forename",StringType(),True),
                               StructField("surname",StringType(),True)
                               ]
                       )
drivers_schema=StructType(fields=[StructField("code",StringType(),True),
                                  StructField("dob",DateType(),True),
                                  StructField("driverId",IntegerType(),True),
                                  StructField("name",name_schema),
                                  StructField("nationality",StringType(),True),
                                  StructField("number",IntegerType(),True),
                                  StructField("url",StringType(),True),
                                  StructField("driverRef",StringType(),True)
                                  ]
                          )

#### Apply Schema

In [0]:
df=spark.read.schema(drivers_schema).json(f"{raw_folder_path}/{v_file_date}/drivers.json")

#### Rename and Add New Columns

In [0]:
from pyspark.sql.functions import col,lit,concat,current_timestamp
renamed_df=df.withColumnRenamed("driverId","driver_id")\
.withColumnRenamed("driverRef","driver_ref")\
.withColumn("name",concat(col("name.forename"),lit(" "),col('name.surname')))\
.withColumn("data_source",lit(v_data_source))\
.withColumn("file_date",lit(v_file_date))
renamed_df=add_ingestion_date(renamed_df)

#### Drop url column

In [0]:
final_df=renamed_df.drop("url")

#### Write Data to Dl in parquet format

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")